<a href="https://colab.research.google.com/github/hakim733/AI-project/blob/main/Optimizing_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
!pip install keras_tuner

In [109]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

here we are going to see if keras is compatible with tensorflow

In [110]:
print(tf.__version__)

2.18.0


loading the keras dataset

In [111]:
fashion_mnist=keras.datasets.fashion_mnist

#Pre-processing
splitting into train and test

In [112]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()


Next we are going to scale the data to be between 0 and 1, so it will help us to quickly create our models

In [123]:
train_images= train_images/255.0
test_images=test_images/255.0
train_images.min(), test_images.max()

(np.float64(0.0), np.float64(0.00392156862745098))

Reshape the data

In [114]:
train_images[0].shape


(28, 28)

In [115]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

Building the model

In [122]:
def build_model(hp):  #hp-hyperparameter
  model = keras.Sequential([
    keras.layers.Conv2D(#convolution 2d array
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),#my filter
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),#aka filter size, choice--select range of parameter
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(#2nd convo 2d layer
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),#flatten my layer
    keras.layers.Dense(#dense layer
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),#minvlaue=noof nodes(btw 32-128) to be selected
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')#last dense layer
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),#when we use adam we proveide learning rate with value
              loss='sparse_categorical_crossentropy',#
              metrics=['accuracy'])

  return model










NameError: name 'Model' is not defined

In [ ]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective="val_accuracy",
                          max_trials=5,directory="output",project_name="Mnist Fashion1",)

In [ ]:
tuner_search.search(train_images,train_labels,epochs=1,validation_split=0.1)

In [124]:
model=tuner_search.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [125]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 80)     │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 32)     │        23,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │       589,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 614,058 (2.34 MB)

 Trainable params: 614,058 (2.34 MB)

 Non-trainable params: 0 (0.00 B)

Build the model


In [127]:

history= model.fit(train_images, train_labels, epochs=50, validation_split=0.2)

Epoch 1/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 169s 112ms/step - accuracy: 0.1007 - loss: 2.3101 - val_accuracy: 0.0995 - val_loss: 2.3028
Epoch 2/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 164s 109ms/step - accuracy: 0.0976 - loss: 2.3028 - val_accuracy: 0.1005 - val_loss: 2.3027
Epoch 3/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 203s 110ms/step - accuracy: 0.0993 - loss: 2.3028 - val_accuracy: 0.0983 - val_loss: 2.3028
Epoch 4/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 201s 109ms/step - accuracy: 0.1004 - loss: 2.3027 - val_accuracy: 0.0983 - val_loss: 2.3029
Epoch 5/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 202s 109ms/step - accuracy: 0.0979 - loss: 2.3028 - val_accuracy: 0.0957 - val_loss: 2.3029
Epoch 6/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 196s 105ms/step - accuracy: 0.1020 - loss: 2.3028 - val_accuracy: 0.0983 - val_loss: 2.3028
Epoch 7/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 161s 108ms/step - accuracy: 0.0993 - loss: 2.3028 - val_accuracy: 0.0957 - val_loss: 2.3028
Epoch 8/50
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 163s 109ms/step - ac

In [128]:
model

<Sequential name=sequential, built=True>

In [130]:
import numpy as np

sample_input = np.array([[1.0, 2.0, 3.0, 4.0]])  # Replace with actual input shape
prediction = model.predict(sample_input)
print(prediction)

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(1, 4), dtype=float32). Expected shape (None, 28, 28, 1), but input has incompatible shape (1, 4)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 4), dtype=float32)
  • training=False
  • mask=None